# Imports

In [1]:
# parser
from bs4 import BeautifulSoup
# data
import pandas as pd
import numpy as np
# requests
import requests
import os
# random
import random
# error mute
import contextlib

# 1. Codeup Blog Articles

In [2]:
# visually inspecting, in order to pull article titles it will be an h2
url = 'https://codeup.edu/blog/'

### Header Randomization

In [3]:
# 
user_agents = [
  "Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0",
  "Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0",
  "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
  'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36',
  'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
  'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36,gzip(gfe)'
  ]
random_user_agent = random.choice(user_agents)

headers = {'User-Agent': random_user_agent}

response = requests.get(url, headers=headers)

In [4]:
# response status code
print(response)

# response text
response.text

<Response [200]>


'<!DOCTYPE html>\n<html lang="en-US">\n<head>\n\t<meta charset="UTF-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<link rel="pingback" href="https://codeup.edu/xmlrpc.php" />\n\n\t<script type="text/javascript">\n\t\tdocument.documentElement.className = \'js\';\n\t</script>\n\t\n\t<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin /><script id="diviarea-loader">window.DiviPopupData=window.DiviAreaConfig={"zIndex":1000000,"animateSpeed":400,"triggerClassPrefix":"show-popup-","idAttrib":"data-popup","modalIndicatorClass":"is-modal","blockingIndicatorClass":"is-blocking","defaultShowCloseButton":true,"withCloseClass":"with-close","noCloseClass":"no-close","triggerCloseClass":"close","singletonClass":"single","darkModeClass":"dark","noShadowClass":"no-shadow","altCloseClass":"close-alt","popupSelector":".et_pb_section.popup","initializeOnEvent":"et_pb_after_init_modules","popupWrapperClass":"area-outer-wrap","fullHeightClass":"full-height","openPopupClass

In [5]:
# soup object initiated
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
# setting blog titles var
blog_titles = soup.select('h2')
list_blog = list(blog_titles)

In [7]:
# location of urls within soup object
blog_titles[1].a['href']

'https://codeup.edu/featured/women-in-tech-panelist-spotlight/'

In [8]:

# retrieving blog names as list
blog_dict = {}
for i,blog in enumerate(list_blog):
    # ignore IndexError for aesthetics
    with contextlib.suppress(IndexError):
    # extracting titles of blogs
        t = blog_titles[i].text
        # list comprehension to retrieve urls
        urls = [title.a['href'] for title in blog_titles if title.a]
        url = urls[i]
        blog_dict.__setitem__(t,url)

# dict
blog_dict


{'Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa': 'https://codeup.edu/featured/apida-heritage-month/',
 'Women in tech: Panelist Spotlight – Magdalena Rahn': 'https://codeup.edu/featured/women-in-tech-panelist-spotlight/',
 'Women in tech: Panelist Spotlight – Rachel Robbins-Mayhill': 'https://codeup.edu/featured/women-in-tech-rachel-robbins-mayhill/',
 'Women in Tech: Panelist Spotlight – Sarah Mellor': 'https://codeup.edu/codeup-news/women-in-tech-panelist-spotlight-sarah-mellor/',
 'Women in Tech: Panelist Spotlight – Madeleine Capper': 'https://codeup.edu/events/women-in-tech-madeleine/',
 'Black Excellence in Tech: Panelist Spotlight – Wilmarie De La Cruz Mejia': 'https://codeup.edu/codeup-news/panelist-spotlight-4/'}

In [9]:
# blog names can be zipped with blog content once defined
codeup_blog = pd.DataFrame([blog_dict]).T.reset_index().rename(columns={'index':'title',0:'url'})
codeup_blog

,title,url
0,Spotlight on APIDA Voices: Celebrating Heritag...,https://codeup.edu/featured/apida-heritage-month/
1,Women in tech: Panelist Spotlight – Magdalena ...,https://codeup.edu/featured/women-in-tech-pane...
2,Women in tech: Panelist Spotlight – Rachel Rob...,https://codeup.edu/featured/women-in-tech-rach...
3,Women in Tech: Panelist Spotlight – Sarah Mellor,https://codeup.edu/codeup-news/women-in-tech-p...
4,Women in Tech: Panelist Spotlight – Madeleine ...,https://codeup.edu/events/women-in-tech-madele...
5,Black Excellence in Tech: Panelist Spotlight –...,https://codeup.edu/codeup-news/panelist-spotli...


## Retrieving content of all urls

In [10]:
# compiling code for 1 article, to see if it will be the same for the others
url_apida = 'https://codeup.edu/featured/apida-heritage-month/'
response = requests.get(url_apida, headers= headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [11]:
content = soup.select('p', class_='entry-content')

In [12]:
content[0].text

'May 24, 2023 | Featured'

In [296]:
# now that it has been determined that the content of every blog post is within the same schema, run a for loop to save it all -- could probably be done in a function
read = []
for url in codeup_blog['url']:
    print(f'This is the URL USED: {url}')
    response = requests.get(url, headers= headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.select('p', class_='entry-content')
    
    for i,c in enumerate(content):
        word = content[i].text
        read.append(word)

# indexing the new list and allocating to blog content list
blog_content = []

This is the URL USED: https://codeup.edu/featured/apida-heritage-month/
This is the URL USED: https://codeup.edu/featured/women-in-tech-panelist-spotlight/
This is the URL USED: https://codeup.edu/featured/women-in-tech-rachel-robbins-mayhill/
This is the URL USED: https://codeup.edu/codeup-news/women-in-tech-panelist-spotlight-sarah-mellor/
This is the URL USED: https://codeup.edu/events/women-in-tech-madeleine/
This is the URL USED: https://codeup.edu/codeup-news/panelist-spotlight-4/


In [14]:
# manually indexing blog contents
article1 = read[0:26]
article2 = read[38:45]
article3 = read[57:65]
article4 = read[77:83]
article5 = read[95:101]
article6 = read[113:121]

In [15]:
blog_content = [article1,article2,article3,article4,article5,article6]

In [16]:
codeup_blog['content'] = blog_content
codeup_blog.content = codeup_blog.content.astype(str)

In [261]:
codeup_blog

,title,url,content
0,Spotlight on APIDA Voices: Celebrating Heritag...,https://codeup.edu/featured/apida-heritage-month/,"['May 24, 2023 | Featured', 'May is traditiona..."
1,Women in tech: Panelist Spotlight – Magdalena ...,https://codeup.edu/featured/women-in-tech-pane...,"['Mar 28, 2023 | Events, Featured', 'Codeup is..."
2,Women in tech: Panelist Spotlight – Rachel Rob...,https://codeup.edu/featured/women-in-tech-rach...,"['Mar 20, 2023 | Events, Featured', 'Codeup is..."
3,Women in Tech: Panelist Spotlight – Sarah Mellor,https://codeup.edu/codeup-news/women-in-tech-p...,"['Mar 13, 2023 | Codeup News, Featured', 'Code..."
4,Women in Tech: Panelist Spotlight – Madeleine ...,https://codeup.edu/events/women-in-tech-madele...,"['Mar 6, 2023 | Events, Featured', 'Codeup is ..."
5,Black Excellence in Tech: Panelist Spotlight –...,https://codeup.edu/codeup-news/panelist-spotli...,"['Feb 16, 2023 | Codeup News, Events, Featured..."


In [ ]:
# get codeup function

def get_codeup():
    '''
    Returns codeup blog articles.
    '''
    # retrieving blog names as list
    blog_dict = {}
    # url/ response/ soup
    url = 'https://codeup.edu/blog/'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # list of blog titles
    blog_titles = soup.select('h2')
    list_blog = list(blog_titles)

    for i,blog in enumerate(list_blog):
    # ignore IndexError for aesthetics
        with contextlib.suppress(IndexError):
        # extracting titles of blogs
            t = blog_titles[i].text
            # list comprehension to retrieve urls
            urls = [title.a['href'] for title in blog_titles if title.a]
            url = urls[i]
            blog_dict.__setitem__(t,url)

    # setting df
    codeup_blog = pd.DataFrame([blog_dict]).T.reset_index().rename(columns={'index':'title',0:'url'})

    # now that it has been determined that the content of every blog post is within the same schema, run a for loop to save it all -- could probably be done in a function
    read = []
    for url in codeup_blog['url']:
        print(f'This is the URL USED: {url}')
        response = requests.get(url, headers= headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.select('p', class_='entry-content')

        # pulling every line
        for i, c in enumerate(content):
            word = content[i].text
            read.append(word)

    # manually setting every description for post - yes there is a better way
    article1 = read[0:26]
    article2 = read[38:45]
    article3 = read[57:65]
    article4 = read[77:83]
    article5 = read[95:101]
    article6 = read[113:121]

    # article body list
    blog_content = [article1,article2,article3,article4,article5,article6]

    # body content column
    codeup_blog['content'] = blog_content
    codeup_blog.content = codeup_blog.content.astype(str)

    return codeup_blog

# 2. News Articles

In [82]:
# compiling code for 1 article, to see if it will be the same for the others
url = 'https://inshorts.com/en/read/business'
response = requests.get(url, headers= headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [126]:
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><title>Short English &amp; Hindi News|Latest News, India News, Current Affairs|Inshorts</title><link href="https://ajax.googleapis.com" rel="dns-prefetch"/><link href="https://assets.inshorts.com" rel="dns-prefetch"/><link href="https://static.inshorts.com" rel="dns-prefetch"/><link href="https://cdn.ampproject.org" rel="dns-prefetch"/><meta content="#ffffff" name="theme-color"/><link href="/assets/images/favicon.png" rel="shortcut icon" type="image/x-icon"><meta content="text/html; charset=utf-8" http-equiv="Content-Type"><meta content="width=device-width,initial-scale=1" name="viewport"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><link href="/assets/images/logo_192.png" rel="icon" sizes="192x192"><link href="https://inshorts.com" rel="canonical">
<link href="https://inshorts.com" rel="alternate">
<meta content="read latest news with inshorts in less than 60 words related to business sports bollywood and technolog

In [143]:
# <span itemprop="headline" class="S2DdZEgzkqC9bYeTJUGw">HDFC Bank's Vigil Aunty ad gets criticism for stop-sign bindi, people call it 'anti-Hindu'</span>

headline = soup.find_all('span', attrs={'itemprop':'headline'})
headline[0].text

"HDFC Bank's Vigil Aunty ad gets criticism for stop-sign bindi, people call it 'anti-Hindu'"

In [ ]:
# checking functionality

categories = ['business','technology','sports','entertainment']
soups = []
articles = []
content = []

for u in categories:
    url = f'https://inshorts.com/en/read/{u}'
    print(f'Saving soup for: {url}')
    response = requests.get(url, headers= headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    soups.append(soup)

    # looping through our soups with their respective category
for s, cat in zip(soups, categories):
        headline = s.find_all('span', attrs={'itemprop':'headline'})
        body = s.find_all('div', attrs={'itemprop':'articleBody'})
        for i, head in enumerate(headline):
            print(f'Appending {cat} website {i}th headline / body')
            articles.append({'title': headline[i].text, 'category': cat, 'body': body[i].text})
            

In [290]:
def get_inshorts():
    '''
    Iterably retrieves the Business, Technology, Sports & Entertainment portion of inshorts articles.
    '''
    base_url = 'https://inshorts.com/en/read/'
    categories = ['business','technology','sports','entertainment']
    soups = []
    articles = []

    # retrieving the soup blurbs from url cycling for category subpage
    for u in categories:
        url = base_url + u
        # randomized user agents along with our iterated urls
        response = requests.get(url, headers= headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        soups.append(soup)
        print(f'Saving soup for: {url}')

    # looping through soups with their respective category
    for s, cat in zip(soups, categories):
            headline = s.find_all('span', attrs={'itemprop':'headline'})
            body = s.find_all('div', attrs={'itemprop':'articleBody'})
            for i, h in enumerate(headline):
                # h is unused just needed the enumaration to iterate and count
                print(f'Appending {cat} website {i}th headline / body')
                articles.append({'title': headline[i].text, 'category': cat, 'body': body[i].text})
                print(f'{headline[i].text}, {body[i].text}\n\n+ + + + + + + + + +\n')

    articles_df = pd.DataFrame(articles)
    return articles_df
            
        

In [289]:
# checking functionality

art = get_inshorts()

Saving soup for: https://inshorts.com/en/read/business
Saving soup for: https://inshorts.com/en/read/technology
Saving soup for: https://inshorts.com/en/read/sports
Saving soup for: https://inshorts.com/en/read/entertainment
Appending business website 0th headline / body
Victoria's Secret ex-CEO cuts Harvard ties for not backing Israel, Victoria's Secret ex-CEO Leslie Wexner's foundation announced it's cutting its "financial and programmatic" ties with Harvard. "We are stunned and sickened at the dismal failure of Harvard's leadership to take a clear...stand against the barbaric murders of innocent Israeli civilians," The Wexner Foundation said. Harvard's leaders were "tiptoeing" over Hamas' attacks against Israel, it added.
+ + + + + + + + + +

Appending business website 1th headline / body
HDFC Bank's Vigil Aunty ad gets criticism for stop-sign bindi, people call it 'anti-Hindu', HDFC Bank's latest advertisement featuring Vigil Aunty is receiving criticism online. The advertisement, 

In [297]:
art

,title,category,body
0,Victoria's Secret ex-CEO cuts Harvard ties for...,business,Victoria's Secret ex-CEO Leslie Wexner's found...
1,HDFC Bank's Vigil Aunty ad gets criticism for ...,business,HDFC Bank's latest advertisement featuring Vig...
2,IMEC big opportunity for investors to partner ...,business,PM Narendra Modi at the Global Maritime India ...
3,"ICICI Bank fined ₹12 crore, Kotak ₹3.95 crore ...",business,The Reserve Bank of India (RBI) has imposed a ...
4,35 lakh weddings in 23 days to generate record...,business,Traders' body Confederation of All India Trade...
5,"Mahadev app key accused arrives from Dubai, he...",business,"Mrugank Mishra, a key accused in Mahadev betti..."
6,Bankman-Fried's trial delay request over Adder...,business,A US court denied FTX Founder Sam Bankman-Frie...
7,Indian wheat prices hit 8-month high,business,Prices of wheat in India have reached an eight...
8,Dabur gets ₹321 crore GST demand notice,business,Dabur India has received a notice to pay Goods...
9,'Kill list' of LinkedIn staff being fired was ...,business,A list with names of about 500 employees was l...
